In [6]:
# -*- coding: utf-8 -*-
#CNVD爬虫
import requests
from lxml import etree
import csv
import time
import random
from collections import OrderedDict
import codecs
from datetime import date
from multiprocessing.dummy import Pool as Threadpool
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.expected_conditions import alert_is_present
from selenium.webdriver.common.alert import Alert
import ast
import json
import re
import os
import string
import cookielib
import urllib2

# 判断是否爬取页面信息，为1则重新爬并缓存页面，为0则直接从缓存页面捕获信息。
is_crwal = 0

class Wenkuspider(object):
    def __init__(self):
        self.__file__ = r'C:/Users/wz/Desktop/lvmeng/paloalt_submit00.json'
        self.__link__ = r"https://openapi.book118.com/login/login/index.html"
        self.__loadeddir__ = 'loadedhtml0'
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
        # 如果从某处断线了，可以更改起始的url地址
        #self.start_url = "http://www.cnvd.org.cn/flaw/list.htm"
        self.count = 0
        self.cookies = self.get_cookies()

    # 保存网页源码到本地，local_filename表示当前页面的完整名称
    def download_document(self, url, local_filename, retry=3):
#         print(url)
#         print(local_filename)
        local_dirname = os.path.dirname(local_filename)
        if not os.path.isdir(local_dirname):
            print "create dir:", local_dirname
            os.makedirs(local_dirname)
        req = urllib2.Request(url)
        req.add_header('User-Agent',
                       'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3135.5 Safari/537.36')
        retry_init = retry
        while retry:
            try:
                resp = urllib2.urlopen(req, timeout=30)
                content = resp.read()
                f = open(local_filename, "w")
                f.write(content)
                f.close()
                break
            except Exception, e:
                print e
                retry = retry - 1
                print "retry %s after %ss ..." % (url, retry_init - retry)
                sleep(retry_init - retry)
    
    # 获取网站cookie
    def get_cookies(self):
        # 构建一个CookieJar对象实例来保存cookie
        cookiejar = cookielib.CookieJar()
        # 使用HTTPCookieProcessor()来创建cookie处理器对象，参数为CookieJar()对象
        handler=urllib2.HTTPCookieProcessor(cookiejar)
        # 通过 build_opener() 来构建opener
        opener = urllib2.build_opener(handler)
        # 4. 以get方法访问页面，访问之后会自动保存cookie到cookiejar中
        opener.open(self.__link__)
        # 可以按标准格式将保存的Cookie打印出来
        cookieStr = ''
        for item in cookiejar:
            cookieStr += "'"+item.name + "':'" + item.value + "',"
        cookie = ast.literal_eval('{'+cookieStr+'}')
#         print(cookie)
        return cookie
    # get请求
    def parse(self, i):
#         time.sleep(random.randint(5, 10))
        self.count += 1
        print('每两个请求重新获取cookie，重置完成后，现在是第%s个'%str(self.count))
        if(self.count == 2):
            self.cookies = self.get_cookies()
            self.count = 0
        url=self.__link__+'/?page=%s&sort=date'%str(i)
        print(1)
        print(url)
        html = requests.get(url, headers=self.headers,
                            cookies=self.cookies).content
#         print(html)
        html = etree.HTML(html)
#         print('第%s页'%str(int(i/20+1)))
#         print(html)
        return html

    # get请求网页具体信息
    def parsedetail(self, url):
#         time.sleep(random.randint(5, 10))
        self.count += 1
        print('每两个请求重新获取cookie，重置完成后，现在是第%s个'%str(self.count))
        if(self.count == 2):
            self.cookies = self.get_cookies()
            self.count = 0
        html = requests.get(url, headers=self.headers,
                            cookies=self.cookies).content
#         print(html)
        html = etree.HTML(html)
#         print('第%s页'%str(int(i/20+1)))
#         print(html)
        return html

    # 对第i页网页列表逐个进行解析获取细节数据
    def get_list_url(self, html):
        # xpath具备较多缺陷，当在谷歌xpath插件上构建成功后，可以考虑删除tbody作为当前脚本的构建结果
        list_url = html.xpath('//*[@id="chartForm"]/div/div[2]/table//tr/td[2]/a/@href')  
#         print(list_url)
        for url0 in list_url:
            url = self.__link__ + url0
            self.parse_detail(url)

    # 解析缓存网页的详细信息
    def parse_loadedhtml(self, html,url):
        # item = OrderedDict()  # 如果要存入csv文档，建议用有序字典
        item = OrderedDict()
        # 获取cnvd id
        list_url=html.xpath('/html/head/title/text()')[0]
        # list_url.split()[0]
        if 'CVE' in list_url.split()[0]:
            item["app_name"] = 'Palo Alto Networks'
            item["cve_list"] = list_url.split()[0]
            
            # 获取漏洞影响的产品版本号
            list_version = html.xpath(
                '//table[@class="neat card"]//td/text()')
            # list_version
            # ''.join(list_version)
            versions={}
            if 'Windows' not in ''.join(list_version) and 'OS X' not in ''.join(list_version):
                for i in range(len(list_version)):
                    if '<' in list_version[i]:
                        versions[list_version[i-1]]=''.join(list(filter(lambda ch: ch in '0123456789.', list_version[i])))
                item["packages"] = versions
#                 print(item["packages"])
            
                # 获取sa_id
                item["sa_id"] = html.xpath(
                    "//h4/text()")
                if 'PAN-OS' in item["sa_id"][0]:
                    item["sa_id"] = "_".join((item["cve_list"],item["sa_id"][0]))

                    # 获得sa_link
                    item["sa_link"] = url

                    # 获得sa_title
                    item["sa_title"] = html.xpath('/html/head/title/text()')
                    item["sa_title"] = item["sa_title"][0]

                    # 获得sa_url
                    item["sa_url"] = url
                    
                    # 保存为json格式
                    item = [item]
                    # 输出：[OrderedDict([('target', 'total_result'), ('key1', 'value1'), ('key2', 'value2'), ('key3', 'value3')])]
                    current_dir = os.path.dirname(self.__file__)
                    path = os.path.join(os.path.join(current_dir), self.__file__.split('/')[-1])
#                     print(3)
#                     print(path)
#                     input()
                    with open(path, 'a') as f:
                        json.dump(item, f, encoding="utf-8", ensure_ascii=False, indent=4, separators=(',', ':'))
    
    # 获取详细漏洞信息数据，采用get请求方法
    def parse_detail(self, url):
#         time.sleep(random.randint(5, 10))
        html = self.parsedetail(url)
        # item = OrderedDict()  # 如果要存入csv文档，建议用有序字典
        item = OrderedDict()
        
        # 获取cnvd id
        list_url=html.xpath('/html/head/title/text()')[0]
        # list_url.split()[0]
        if 'CVE' in list_url.split()[0]:
            item["app_name"] = 'Palo Alto Networks'
            item["cve_list"] = list_url.split()[0]
            
            # 获取漏洞影响的产品版本号
            list_version = html.xpath(
                '//table[@class="neat card"]//td/text()')
            # list_version
            # ''.join(list_version)
            versions={}
            if 'Windows' not in ''.join(list_version) and 'OS X' not in ''.join(list_version):
                for i in range(len(list_version)):
                    if '<' in list_version[i]:
                        versions[list_version[i-1]]=''.join(list(filter(lambda ch: ch in '0123456789.', list_version[i])))
                item["packages"] = versions
#                 print(item["packages"])
            
                # 获取sa_id
                item["sa_id"] = html.xpath(
                    "//h4/text()")
                if 'PAN-OS' in item["sa_id"][0]:
                    item["sa_id"] = "_".join((item["cve_list"],item["sa_id"][0]))

                    # 获得sa_link
                    item["sa_link"] = url

                    # 获得sa_title
                    item["sa_title"] = html.xpath('/html/head/title/text()')
                    item["sa_title"] = item["sa_title"][0]

                    # 获得sa_url
                    item["sa_url"] = url
                    
                    # 保存为json格式
                    item = [item]
                    # 输出：[OrderedDict([('target', 'total_result'), ('key1', 'value1'), ('key2', 'value2'), ('key3', 'value3')])]
                    current_dir = os.path.dirname(self.__file__)
                    path = os.path.join(os.path.join(current_dir), self.__file__.split('/')[-1])
                                        
                    print(url)
                    local_filenames = os.path.join(os.path.dirname(self.__file__), self.__loadeddir__)
                    local_filenames = local_filenames+'/'+url.split('/')[-1]+'.html'
                    print(local_filenames) #C:/Users/wz/Desktop/lvmeng\loadedhtml
#                     input()
                    self.download_document(url, local_filenames,1)
                    
                    with open(path, 'a') as f:
                        json.dump(item, f, encoding="utf-8", ensure_ascii=False, indent=4, separators=(',', ':'))

    def run(self):        
        chrome_options = Options()
        # 加上下面两行，解决报错
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
#         chrome_options.add_argument("--headless")
        chrome_options.binary_location =  r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
        driver = webdriver.Chrome(chrome_options=chrome_options)
        # 设置每次ChromeDriver访问的初始页面用来更新cookie，以绕过cnvd爬虫限制 OK
        driver.get("https://openapi.book118.com/login/login/index.html")
        time.sleep(3)
#         driver.set_window_size(1920, 1080)
        raw_input()
        # 进入“我的文档”
        driver.find_element_by_xpath("//ul[@id='nav']/li[4]/a/span").click()
        time.sleep(3)
        # 进入“系统失败可发布”
        driver.find_element_by_xpath('//*[@id="broadside"]/div[1]/ul/li[3]/ul/li[5]/a/em').click()
        time.sleep(3)
#         # 调整网页尺寸
#         driver.execute_script("document.body.style.zoom='0.5'") #缩小
#         time.sleep(3)
        # 切换到frame框架
        driver.switch_to.frame('doc_index_iframe')
#         time.sleep(3)
        # 动态页面中的内容
        zicontent = driver.page_source
        html = etree.HTML(zicontent)
        # 50个文件名组成的列表
        names = html.xpath('//*[@id="table"]/div[1]/table/tbody/tr/td[3]/a/text()')
        num = 1
        num0 = 1
        time.sleep(3)
#         print(num)
        for name in names:
            name = name.strip()
            # 出现弹窗
            xpatha = '#table > div.container > table > tbody > tr:nth-child('
            xpathb = str(num0)
            xpathc = ') > td.col-operate.col-colspan > ul > li:nth-child(3) > a'
            # 点开“修改”按钮
            trial = driver.find_element_by_css_selector(xpatha+xpathb+xpathc)
            driver.execute_script('arguments[0].click()', trial)
            time.sleep(2)
            driver.switch_to_default_content()
            
            time.sleep(0.5)
            # 切换到frame框架layui-layer-iframe2
            framename = 'layui-layer-iframe'+str(int(num))
            driver.switch_to.frame(framename)
            driver.find_element_by_name('keywords').click()     #先点击输入文本的输入框
#             time.sleep(2)
            driver.find_element_by_name('keywords').send_keys(name)
            
#             time.sleep(2)layui-layer-iframe4
#             driver.find_element_by_id('summary').click()     #先点击输入文本的输入框
#             time.sleep(2)
#             driver.find_element_by_id('summary').send_keys(name)
            time.sleep(0.5)            
            driver.find_element_by_xpath('//*[@id="edit_btn_sub"]').click()
            time.sleep(2) 
            driver.switch_to_default_content()
            # 切换到frame框架
            driver.switch_to.frame('doc_index_iframe')
            time.sleep(1)
            # 点开“申请审核按钮”按钮
            xpatha = '#table > div.container > table > tbody > tr:nth-child('
            xpathb = str(num0)
            xpathc = ') > td.col-operate.col-colspan > ul > li:nth-child(1) > a'            
            trial = driver.find_element_by_css_selector(xpatha+xpathb+xpathc)
            driver.execute_script('arguments[0].click()', trial)
            time.sleep(2)
            driver.switch_to_default_content()
            # 切换到frame框架layui-layer-iframe2
            num = num+2
            framename = 'layui-layer-iframe'+str(int(num))
            driver.switch_to.frame(framename) # layui-layer-iframe9
            driver.find_element_by_name('summary').click()     #先点击输入文本的输入框
#             time.sleep(2)
            driver.find_element_by_name('summary').send_keys(name+name+name+name)
            time.sleep(0.5)            
            driver.find_element_by_xpath('//*[@id="change_audit_btn_sub"]').click()
            time.sleep(2) 
            driver.switch_to_default_content()
            # 切换到frame框架
            driver.switch_to.frame('doc_index_iframe')
            time.sleep(2)
#             time.sleep(5)
#             driver.find_element_by_xpath(xpatha+xpathb+xpathc).click()
            num = num + 2
            num0 = num0+1
            print(num0)
            print(num)        
#         time.sleep(2) 
#         driver.switch_to_default_content()
#         # 切换到frame框架layui-layer-iframe2
# #         num = num+2
#         framename = 'layui-layer'+str(int(num))
#         driver.switch_to.frame(framename) # layui-layer-iframe9
#         time.sleep(0.5)            
#         driver.find_element_by_xpath('//*[@id="change_audit_btn_sub"]').click()
#         time.sleep(2) 
        raw_input()        
        time.sleep(3)
        driver.switch_to_default_content()
        # 切换到frame框架
        driver.switch_to.frame('doc_index_iframe')
        # 动态页面中的内容
        zicontent = driver.page_source
        html = etree.HTML(zicontent)
        # 50个文件名组成的列表
        names = html.xpath('//*[@id="table"]/div[1]/table/tbody/tr/td[3]/a/text()')
        num = 202
        num0 = 1        
        time.sleep(3)
#         print(num)
        for name in names:
            name = name.strip()
            # 出现弹窗
            xpatha = '#table > div.container > table > tbody > tr:nth-child('
            xpathb = str(num0)
            xpathc = ') > td.col-operate.col-colspan > ul > li:nth-child(3) > a'
            # 点开“修改”按钮
            trial = driver.find_element_by_css_selector(xpatha+xpathb+xpathc)
            driver.execute_script('arguments[0].click()', trial)
            time.sleep(2)
            driver.switch_to_default_content()
            
            time.sleep(0.5)
            # 切换到frame框架layui-layer-iframe2
            framename = 'layui-layer-iframe'+str(int(num))
            driver.switch_to.frame(framename)
            driver.find_element_by_name('keywords').click()     #先点击输入文本的输入框
#             time.sleep(2)
            driver.find_element_by_name('keywords').send_keys(name)
            
#             time.sleep(2)layui-layer-iframe4
#             driver.find_element_by_id('summary').click()     #先点击输入文本的输入框
#             time.sleep(2)
#             driver.find_element_by_id('summary').send_keys(name)
            time.sleep(0.5)            
            driver.find_element_by_xpath('//*[@id="edit_btn_sub"]').click()
            time.sleep(2) 
            driver.switch_to_default_content()
            # 切换到frame框架
            driver.switch_to.frame('doc_index_iframe')
            time.sleep(1)
            # 点开“申请审核按钮”按钮
            xpatha = '#table > div.container > table > tbody > tr:nth-child('
            xpathb = str(num0)
            xpathc = ') > td.col-operate.col-colspan > ul > li:nth-child(1) > a'            
            trial = driver.find_element_by_css_selector(xpatha+xpathb+xpathc)
            driver.execute_script('arguments[0].click()', trial)
            time.sleep(2)
            driver.switch_to_default_content()
            # 切换到frame框架layui-layer-iframe2
            num = num+2
            framename = 'layui-layer-iframe'+str(int(num))
            driver.switch_to.frame(framename) # layui-layer-iframe9
            driver.find_element_by_name('summary').click()     #先点击输入文本的输入框
#             time.sleep(2)
            driver.find_element_by_name('summary').send_keys(name+name+name+name)
            time.sleep(0.5)            
            driver.find_element_by_xpath('//*[@id="change_audit_btn_sub"]').click()
            time.sleep(2) 
            driver.switch_to_default_content()
            # 切换到frame框架
            driver.switch_to.frame('doc_index_iframe')
            time.sleep(2)
#             time.sleep(5)
#             driver.find_element_by_xpath(xpatha+xpathb+xpathc).click()
            num = num + 2
            num0 = num0+1
            print(num0)
            print(num)        
#             print(num)
#         print(aa)
        print(num0)
        print(num)
        print('自动审核完毕')
#         driver.quit()
#         raw_input()
#       

if __name__ == "__main__":
    a = Wenkuspider()
    pool = Threadpool(1)  # 单线程跑的慢，但是基本很稳定没有被cnvd封杀。本次跑完整个cnvd库大概十天左右，期间网络波动中断需要维护代码，及爬虫重新爬的页面范围。
    a.run()
    # pool.map(a.run(),self.parse())
    pool.close()
    pool.join()

c:\python27\lib\site-packages\ipykernel_launcher.py:245: DeprecationWarning: use options instead of chrome_options


c:\python27\lib\site-packages\ipykernel_launcher.py:282: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:299: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:310: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:321: DeprecationWarning: use driver.switch_to.default_content instead


2
5
3
9
4
13
5
17
6
21
7
25
8
29
9
33
10
37
11
41
12
45
13
49
14
53
15
57
16
61
17
65
18
69
19
73
20
77
21
81
22
85
23
89
24
93
25
97
26
101
27
105
28
109
29
113
30
117
31
121
32
125
33
129
34
133
35
137
36
141
37
145
38
149
39
153
40
157
41
161
42
165
43
169
44
173
45
177
46
181
47
185
48
189
49
193
50
197
51
201



c:\python27\lib\site-packages\ipykernel_launcher.py:342: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:364: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:381: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:392: DeprecationWarning: use driver.switch_to.default_content instead
c:\python27\lib\site-packages\ipykernel_launcher.py:403: DeprecationWarning: use driver.switch_to.default_content instead


2
206
3
210
4
214
5
218
6
222
7
226
8
230
9
234
10
238
11
242
12
246
13
250
14
254
15
258
16
262
17
266
18
270
19
274
20
278
21
282
22
286
23
290
24
294
25
298
26
302
27
306
28
310
29
314
30
318
31
322
32
326
33
330
34
334
35
338
36
342
37
346
38
350
39
354
40
358
41
362
42
366
43
370
44
374
45
378
46
382
47
386
48
390
49
394
50
398
51
402
51
402
自动审核完毕


In [4]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [5]:
name = 'ddd'
name+name+name+name

'dddddddddddd'